In [1]:
import numpy as np
import pandas as pd
import joblib
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn import metrics

%matplotlib inline

In [2]:
def tl_features(tl):
    
    trainf_path      = tl + '/' + tl + '_train_features.pkl'
    validationf_path = tl + '/' + tl + '_validation_features.pkl'
    testf_path       = tl + '/' + tl + '_test_features.pkl'
    trainl_path      = tl + '/' + tl + '_train_labels.pkl'
    validationl_path = tl + '/' + tl + '_validation_labels.pkl'
    testl_path       = tl + '/' + tl + '_test_labels.pkl'
    
    train_features      = joblib.load(trainf_path)
    validation_features = joblib.load(validationf_path)
    test_features       = joblib.load(testf_path)
    train_labels        = to_categorical(joblib.load(trainl_path))
    validation_labels   = to_categorical(joblib.load(validationl_path))
    test_labels         = to_categorical(joblib.load(testl_path))
    
    return train_features, validation_features, test_features, train_labels, validation_labels, test_labels

In [3]:
def get_metrics(model, test_features, test_labels):
    
    predictions = model.predict_classes(test_features)
    class_map = {'0' : 'air_conditioner', '1' : 'car_horn', '2' : 'children_playing', '3' : 'dog_bark', '4' : 'drilling', 
                 '5' : 'engine_idling', '6' : 'gun_shot', '7' : 'jackhammer', '8' : 'siren', '9' : 'street_music'}
    
    true_labels      = [class_map[str(np.argmax(label))] for label in test_labels]
    predicted_labels = [class_map[str(label)] for label in predictions]
    
    Accuracy  = np.round(metrics.accuracy_score(true_labels, predicted_labels), 4)
    Precision = np.round(metrics.precision_score(true_labels, predicted_labels, average='weighted'), 4)
    Recall    = np.round(metrics.recall_score(true_labels, predicted_labels, average='weighted'), 4)
    F1_Score  = np.round(metrics.f1_score(true_labels, predicted_labels, average='weighted'), 4)
    
    return [Accuracy, Precision, Recall, F1_Score]

In [4]:
def get_model(train_features):
   
    model = models.Sequential()
    model.add(layers.Dense(1024, activation='relu', input_shape=(train_features.shape[1],)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
    return model

In [5]:
def train_model(model, train_features, train_labels, validation_features, validation_labels):
    
    model.fit(train_features, train_labels, epochs=50, batch_size=128,
              validation_data=(validation_features, validation_labels), 
              shuffle=True, verbose=1)

In [4]:
#results = pd.DataFrame()
#results['metrics'] = ['Accuracy', 'Precision', 'Recall', 'F1_Score']

results = pd.read_csv('results.csv')

In [8]:
tl_name_1 = 'Vgg16'
Vgg16_train_features, Vgg16_validation_features, Vgg16_test_features, Vgg16_train_labels, Vgg16_validation_labels, Vgg16_test_labels = tl_features(tl_name_1)
Vgg16_train_features.shape, Vgg16_validation_features.shape, Vgg16_test_features.shape, Vgg16_train_labels.shape, Vgg16_validation_labels.shape, Vgg16_test_labels.shape

((18300, 2048),
 (6100, 2048),
 (6100, 2048),
 (18300, 10),
 (6100, 10),
 (6100, 10))

In [13]:
tl_model_1 = get_model(Vgg16_train_features)
tl_model_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_13 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)              

In [20]:
train_model(tl_model_1, Vgg16_train_features, Vgg16_train_labels, Vgg16_validation_features, Vgg16_validation_labels)

Train on 18300 samples, validate on 6100 samples
Epoch 1/50
18300/18300 [==============================] - 17s 907us/sample - loss: 2.4356 - accuracy: 0.3866 - val_loss: 1.0687 - val_accuracy: 0.6634
Epoch 2/50
18300/18300 [==============================] - 14s 783us/sample - loss: 1.1859 - accuracy: 0.6001 - val_loss: 0.8759 - val_accuracy: 0.7279
Epoch 3/50
18300/18300 [==============================] - 14s 790us/sample - loss: 0.9895 - accuracy: 0.6774 - val_loss: 0.7660 - val_accuracy: 0.7670
Epoch 4/50
18300/18300 [==============================] - 13s 731us/sample - loss: 0.8607 - accuracy: 0.7226 - val_loss: 0.6817 - val_accuracy: 0.7984
Epoch 5/50
18300/18300 [==============================] - 13s 717us/sample - loss: 0.7716 - accuracy: 0.7507 - val_loss: 0.6681 - val_accuracy: 0.8002
Epoch 6/50
18300/18300 [==============================] - 13s 729us/sample - loss: 0.6979 - accuracy: 0.7773 - val_loss: 0.6193 - val_accuracy: 0.8064
Epoch 7/50
18300/18300 [=====================

In [23]:
results['Vgg16'] = get_metrics(tl_model_1, Vgg16_test_features, Vgg16_test_labels)

In [24]:
results

,metrics,Vgg16
0,Accuracy,0.8830
1,Precision,0.8871
2,Recall,0.8830
3,F1_Score,0.8834


In [28]:
tl_name_2 = 'ResNet50'
ResNet50_train_features, ResNet50_validation_features, ResNet50_test_features, ResNet50_train_labels, ResNet50_validation_labels, ResNet50_test_labels = tl_features(tl_name_2)
ResNet50_train_features.shape, ResNet50_validation_features.shape, ResNet50_test_features.shape, ResNet50_train_labels.shape, ResNet50_validation_labels.shape, ResNet50_test_labels.shape

((18300, 8192),
 (6100, 8192),
 (6100, 8192),
 (18300, 10),
 (6100, 10),
 (6100, 10))

In [29]:
tl_model_2 = get_model(ResNet50_train_features)
tl_model_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 1024)              8389632   
_________________________________________________________________
dropout_20 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_21 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_22 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 512)              

In [30]:
train_model(tl_model_2, ResNet50_train_features, ResNet50_train_labels, ResNet50_validation_features, ResNet50_validation_labels)

Train on 18300 samples, validate on 6100 samples
Epoch 1/50
18300/18300 [==============================] - 31s 2ms/sample - loss: 1.4973 - accuracy: 0.4901 - val_loss: 0.8591 - val_accuracy: 0.7097
Epoch 2/50
18300/18300 [==============================] - 27s 1ms/sample - loss: 0.8988 - accuracy: 0.7083 - val_loss: 0.6347 - val_accuracy: 0.7905
Epoch 3/50
18300/18300 [==============================] - 28s 2ms/sample - loss: 0.7160 - accuracy: 0.7748 - val_loss: 0.5491 - val_accuracy: 0.8233
Epoch 4/50
18300/18300 [==============================] - 27s 1ms/sample - loss: 0.6172 - accuracy: 0.8073 - val_loss: 0.4908 - val_accuracy: 0.8434
Epoch 5/50
18300/18300 [==============================] - 27s 1ms/sample - loss: 0.5104 - accuracy: 0.8406 - val_loss: 0.4627 - val_accuracy: 0.8497
Epoch 6/50
18300/18300 [==============================] - 28s 2ms/sample - loss: 0.4849 - accuracy: 0.8486 - val_loss: 0.4433 - val_accuracy: 0.8575
Epoch 7/50
18300/18300 [==============================] -

In [31]:
results['ResNet50'] = get_metrics(tl_model_2, ResNet50_test_features, ResNet50_test_labels)

In [41]:
tl_name_3 = 'Vgg19'
Vgg19_train_features, Vgg19_validation_features, Vgg19_test_features, Vgg19_train_labels, Vgg19_validation_labels, Vgg19_test_labels = tl_features(tl_name_3)
Vgg19_train_features.shape, Vgg19_validation_features.shape, Vgg19_test_features.shape, Vgg19_train_labels.shape, Vgg19_validation_labels.shape, Vgg19_test_labels.shape

((18300, 2048),
 (6100, 2048),
 (6100, 2048),
 (18300, 10),
 (6100, 10),
 (6100, 10))

In [42]:
tl_model_3 = get_model(Vgg19_train_features)
tl_model_3.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_24 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_25 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_26 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 512)              

In [43]:
train_model(tl_model_3, Vgg19_train_features, Vgg19_train_labels, Vgg19_validation_features, Vgg19_validation_labels)

Train on 18300 samples, validate on 6100 samples
Epoch 1/50
18300/18300 [==============================] - 10s 560us/sample - loss: 2.9509 - accuracy: 0.3531 - val_loss: 1.2827 - val_accuracy: 0.5920
Epoch 2/50
18300/18300 [==============================] - 8s 443us/sample - loss: 1.3776 - accuracy: 0.5334 - val_loss: 1.0406 - val_accuracy: 0.6713
Epoch 3/50
18300/18300 [==============================] - 8s 439us/sample - loss: 1.1504 - accuracy: 0.6201 - val_loss: 0.8986 - val_accuracy: 0.7195
Epoch 4/50
18300/18300 [==============================] - 8s 450us/sample - loss: 1.0504 - accuracy: 0.6571 - val_loss: 0.8450 - val_accuracy: 0.7389
Epoch 5/50
18300/18300 [==============================] - 8s 448us/sample - loss: 0.9556 - accuracy: 0.6881 - val_loss: 0.8029 - val_accuracy: 0.7590
Epoch 6/50
18300/18300 [==============================] - 8s 453us/sample - loss: 0.8821 - accuracy: 0.7196 - val_loss: 0.7355 - val_accuracy: 0.7659
Epoch 7/50
18300/18300 [==========================

In [44]:
results['Vgg19'] = get_metrics(tl_model_3, Vgg19_test_features, Vgg19_test_labels)

In [47]:
results.to_csv('results.csv')

In [5]:
results

,metrics,Vgg16,ResNet50,Vgg19
0,Accuracy,0.8830,0.9005,0.8387
1,Precision,0.8871,0.9007,0.8425
2,Recall,0.8830,0.9005,0.8387
3,F1_Score,0.8834,0.9002,0.8382
